In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku 
import numpy as np 
import pandas as pd

Definition of a plot function for training result visualization

## Data preproccesing

In [ ]:
# Mounting the google drive to google colab in order to load the data files directly from it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We load a txt file containing Shakespeare sonnets

In [ ]:
data = open('/content/drive/MyDrive/EPITA_NLP/Course3/sonnets.txt').read()

corpus = data.lower().split("\n")

We need to transform each text sentence into token sequence.

Then we need to generate from each token sequence, several token subsequences in order to augment the dataset. **Remember** that we want to learn how to predict the next word of a sentence. A sentence of 5 words can so be used to generate 4 training sequences.

e.g. the sentence "*to be or not to be*" can give the following training sequences: 

"*to* **be**"

"*to be* **or**"

"*to be or* **not**"

"*to be or not* **to**"

"*to be or not to* **be**"

In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

#--- create predictors and label
# The predictors correspond to the input sequence without the last token
# the label corresponds to the last token (the one we want to predict)
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

In [ ]:
sequence_test = tokenizer.texts_to_sequences(["to be or not to be"])[0]
sequence_test

[3, 21, 36, 15, 3, 21]

In [ ]:
pad_sequences([sequence_test], maxlen=max_sequence_len, padding='post')

array([[ 3, 21, 36, 15,  3, 21,  0,  0,  0,  0,  0]], dtype=int32)

In [ ]:
label

array([ 417,  877,  166, ..., 3210,   15,   14], dtype=int32)

## Neural network model definition

Build a neural network using at least one LSTM layer

(you may have a look at https://keras.io/api/layers/)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
______________________________________________

In [ ]:
 history = model.fit(predictors, label, epochs=100, verbose=1, validation_split=0.1 )

Epoch 1/100
435/435 [==============================] - 49s 103ms/step - loss: 6.9220 - accuracy: 0.0216 - val_loss: 6.7533 - val_accuracy: 0.0220
Epoch 2/100
435/435 [==============================] - 44s 100ms/step - loss: 6.4866 - accuracy: 0.0241 - val_loss: 6.9354 - val_accuracy: 0.0220
Epoch 3/100
435/435 [==============================] - 44s 101ms/step - loss: 6.3890 - accuracy: 0.0238 - val_loss: 6.9683 - val_accuracy: 0.0155
Epoch 4/100
435/435 [==============================] - 44s 101ms/step - loss: 6.2765 - accuracy: 0.0295 - val_loss: 7.0989 - val_accuracy: 0.0213
Epoch 5/100
435/435 [==============================] - 44s 101ms/step - loss: 6.1792 - accuracy: 0.0357 - val_loss: 7.2174 - val_accuracy: 0.0388
Epoch 6/100
435/435 [==============================] - 44s 101ms/step - loss: 6.1025 - accuracy: 0.0379 - val_loss: 7.3718 - val_accuracy: 0.0356
Epoch 7/100
435/435 [==============================] - 44s 100ms/step - loss: 6.0297 - accuracy: 0.0421 - val_loss: 7.5063 -

## Result visualization

## Text generation

You can now use your model to sequentially generate new words from a given uncomplete sentence

In [ ]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_aux = model.predict(token_list, verbose=0)
	predicted = np.argmax(predicted_aux, axis=1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Help me Obi Wan Kenobi, you're my only hope of either's pen night worth gentle part rhyme lies impute rage ' and bevel gone so last alone seen shown last alone rage ' doth lust doth share truth dead woe pride ill report skill repair so bright alone see so sort sort bright be growest date prime rage can spend brought still none toil'd lie still seen seen remain bright doth approve another too ill skill skill report ' rage of wrong ' must untold untold alone alone now lust new who lie remain alone doth bold ' hath die my barren rhyme ' doth hell to live alone


Reference:

https://www.coursera.org/learn/natural-language-processing-tensorflow